<a href="https://www.kaggle.com/code/leminhtri2702/sign-language-7f46f2-e90d5b?scriptVersionId=111031687" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import random
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
size = 28
# Tao model MLP
#print("Create MLP model")
#mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation='relu',max_iter=500,warm_start=False,verbose=False)

# Tao list file hinh training
# Get list of training and test images
print("Loading data for training")
from pathlib import Path
train_img_folder = Path('../input/asl-alphabet-test')  # GenderDataset

test_img_folder = Path('../input/asl-alphabet-test')
#test_folder = img_folder / '../input/handdatainput/hand-data/data-test'
#train_folder = img_folder / 'data-train'
#test_images = list(test_folder.glob('**/*.png'))
test_images = list(test_img_folder.glob('**/*.jpg'))
train_images = list(train_img_folder.glob('**/*.jpg'))
#print(test_images)
# Prepare test data
num__test_images = len(test_images)

Loading data for training


In [3]:
labels_test=[]
X_test = np.empty((0,size*size),dtype = np.float32)
Y_test = np.zeros((num__test_images))
for i in np.arange(num__test_images):
    # load image
    image_path = str(test_images[i])
    img = cv.imread(image_path,cv.IMREAD_GRAYSCALE)
    img = cv.resize(img,(size,size))
    # preprocessing: filter, rotate, change brightness...
    # normalize 
    img = img.astype(np.float32)
    cv.normalize(img, img, 0,1,cv.NORM_MINMAX)
    # reshape
    img = img.ravel()
    # stack images
    X_test = np.vstack((X_test,img))
    label=test_images[i].parent.name
    labels_test.append(label)
labels=np.array(labels_test)
onehot = OneHotEncoder()
Y_test = onehot.fit_transform(labels.reshape(-1,1))
print(Y_test.shape)

(1740, 29)


In [4]:
train_list=[]
num__train_images = len(train_images)
X_train = np.empty((0,size*size),dtype = np.float32)
Y_train = np.zeros((num__test_images))
for i in np.arange(num__train_images):
    # load image
    image_path = str(train_images[i])
    img = cv.imread(image_path,cv.IMREAD_GRAYSCALE)
    img = cv.resize(img,(size,size))
    # preprocessing: filter, rotate, change brightness...
    # normalize 
    img = img.astype(np.float32)
    cv.normalize(img, img, 0,1,cv.NORM_MINMAX)
    # reshape
    img = img.ravel()
    # stack images
    X_train = np.vstack((X_train,img))
    train_label=train_images[i].parent.name
    train_list.append(train_label)
train_labels=np.array(train_list)
onehot = OneHotEncoder()
Y_train = onehot.fit_transform(train_labels.reshape(-1,1))
print(Y_train.shape)

(1740, 29)


In [5]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((1740, 784), (1740, 29), (1740, 784), (1740, 29))

In [6]:
print("Create MLP model")
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation='relu',max_iter=800,warm_start=False,verbose=False)

Create MLP model


In [7]:
    # method 1: use partial_fit, set warm_start = False
   # mlp.partial_fit(X_train,Y_train, classes=n_classes)
    # method 2: use fit, set warm_start = True
    mlp = mlp.fit(X_train,Y_train)
    # Evaluation
    # scores_train.append(mlp.score(X, Y))
    # scores_test.append(mlp.score(X_test, Y_test))
    # print('Train accuracy = ',scores_train[-1],'; Test accuracy = ',  scores_test[-1])
    y_pred = mlp.predict(X_test)
    accuracy = metrics.accuracy_score(Y_test, y_pred)
    
    #print('Batch ',b+1,' completed. Accuracy = ', accuracy)

print("Training completed")

Training completed


In [8]:
print(accuracy)

1.0


In [9]:
import pickle
filename = 'mlpModel.sav'
print("Save trained model as:",filename)
pickle.dump(mlp, open(filename, 'wb'))

Save trained model as: mlpModel.sav


In [10]:
import cv2 as cv
x = np.empty((0,size*size),dtype=np.float32)
#load img
img_path = '../input/handdatainput/hand-data/data-test/G/G0.png'
img = cv.imread(img_path,cv.IMREAD_GRAYSCALE)
#preprocessing: filter,rotate,change brightness,..
#normalize
img = cv.resize(img,(size,size))
img = img.astype(np.float32)
cv.normalize(img,img,0,1,cv.NORM_MINMAX)
#reshape
img = img.ravel()
#stack img
x = np.vstack((x,img))
y_pred = mlp.predict(x)
y_pred

<1x29 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [11]:
Y_pred_classes = np.argmax(y_pred,axis = 1)

In [12]:
Y_pred_classes

matrix([[0]])